# Preliminary investigation of the *UHCW* dataset
The [University Hospitals - Coventry & Warwickshire](https://www.uhcw.nhs.uk) (UHCW) has set up an online [portal](https://www.swiftqueue.co.uk/uhcw.php) to facilitate the booking of blood tests appointments in 18 centers around Coventry (UK).  


Available appointments have been recorded in the dataset `appointments.csv`. 
In this notebook we take a first look at this dataset.

## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
%matplotlib inline

## Helper functions

In [2]:
def cartesianProduct(left, right):
    """Returns Cartesian product of dataframe.
    From https://stackoverflow.com/a/53699013/9472676 with some arrangement:
    """
    # Rename columns if "key" already exists.
    if "key" in list(left.columns):
        key_x = "key_x"
    else:
        key_x = ""
    if "key" in list(right.columns):
        key_y = "key_x"
    else:
        key_y = ""
    new_key = hash("".join(list(left.columns) + list(right.columns)))
    left.rename(index=str, columns={"key": str(new_key)}, inplace=True)
    right.rename(index=str, columns={"key": str(new_key)}, inplace=True)
    # Calculate Cartesian product
    df_cartesian = left.assign(key=1).merge(
        right.assign(key=1), on='key'
    ).drop('key', 1)
    # Rename columns back to "key" if they had been changed
    left.rename(index=str, columns={str(new_key): "key"}, inplace=True)
    right.rename(index=str, columns={str(new_key): "key"}, inplace=True)
    if (key_x != "") and (key_y != ""):
        df_cartesian.rename(index=str, columns={str(new_key)+"_x": "key_x"}, inplace=True)
        df_cartesian.rename(index=str, columns={str(new_key)+"_y": "key_y"}, inplace=True)
    else:
        df_cartesian.rename(index=str, columns={str(new_key): "key"}, inplace=True)
    return df_cartesian


In [3]:
def spot_action(df, col='key'):
    """
    Detects booking and cancellation of dataframe.
    
    Parameter
    ---------
    df: dataframe with 0's and 1's
    
    Returns
    -------
    df, modified in place, replacing a transition from 0 to 1 with "cancel",
    a transition from 1 to 0 with "book",
    and "none" otherwise.
    """
    df[col] = df[col] - df[col].shift(1)
    df[col].fillna(method='bfill', inplace=True)
    df[col] = df[col].astype(int)
    df[col] = df[col].apply(lambda delta: "cancel" if delta == -1 else ("book" if delta == 1 else "none"))
    return df

## Data
Read in the `csv` file.

In [4]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
filename = 'sample_appointments.csv'  # appointments.csv
foldername = 'Data/UHCW'
filepath = os.path.join(os.path.expanduser("~"), foldername, filename)

filesize = os.path.getsize(filepath)
if filesize > 1e6:
    print("Size of data file: {}MB.".format(filesize//1000000))
elif filesize > 1e3:
    print("Size of data file: {}KB.".format(filesize//1000))
else:
    print("Size of data file: {}B.".format(filesize))
print("Loading data...")
raw_data = pd.read_csv(filepath, sep=';')
print("Number of records: {0}.".format(raw_data.shape[0]))
print("Column names:\n{}".format("\n".join(raw_data.columns)))

Tuesday 05 February 2019, 22:42:10
Size of data file: 291MB.
Loading data...
Number of records: 5117059.
Column names:
appointment timestamp
center age group
center id
test type
grab timestamp
CPU times: user 2.71 s, sys: 207 ms, total: 2.92 s
Wall time: 2.92 s


Work with a copy `df` of the `raw_data` dataframe.

In [7]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))

df = raw_data.copy()
df = df[['appointment timestamp', 'center id', 'center age group', 'test type', 'grab timestamp']]
df.columns = ['appointment', 'id', 'age group', 'test type', 'grab']
df['appointment'] = pd.to_datetime(df['appointment'])
df['grab'] = pd.to_datetime(df['grab'])
df = df[['id', 'test type', 'age group', 'grab', 'appointment']]

Tuesday 05 February 2019, 22:43:33
CPU times: user 2.1 s, sys: 154 ms, total: 2.25 s
Wall time: 2.25 s


## First observations

In [ ]:
df.info()

There are 18 test centers surving 2 age groups and 5 types of blood tests:

In [ ]:
df[['id', 'age group', 'test type']].nunique()

The age groups are "child" and "adult":

In [ ]:
print(list(df['age group'].unique()))

Each test center serves only one age group but delivers up to three types of blood test:

In [ ]:
df.groupby('id')['age group'].unique()

In [ ]:
df.groupby('id')['test type'].unique()

We can sort the test centers by number of test types:

In [ ]:
df.groupby('id')['test type'].nunique().reset_index().sort_values(['test type', 'id'])

We can sort the centers by their number of records:

In [ ]:
df.groupby('id')['test type'].count().reset_index().sort_values(['test type', 'id'])

We can combine these two in a new dataframe:

In [ ]:
grouped_cid = df.groupby('id')
df1 = grouped_cid['test type'].nunique().reset_index()
df1['count'] = grouped_cid['test type'].count().reset_index()['test type']

df1.sort_values(by=['test type', 'count'])

## Drop *age group* 
We will not need the column `age group`:

In [8]:
df.drop('age group', axis=1, inplace=True)
df.head()

,id,test type,grab,appointment
0,10243,Blood Test,2019-01-15 13:40:00,2019-01-15 14:40:00
1,10243,Blood Test,2019-01-15 13:40:00,2019-01-15 14:45:00
2,10243,Blood Test,2019-01-15 13:40:00,2019-01-15 14:50:00
3,10243,Blood Test,2019-01-15 13:40:00,2019-01-15 14:55:00
4,10243,Blood Test,2019-01-15 13:40:00,2019-01-15 15:10:00


## Sort values for convenience of reading

In [ ]:
df.sort_values(['id', 'test type', 'appointment', 'grab'], inplace=True)

In [ ]:
df.head()

## Restricted dataset
For speed, we will restrict to a smaller subset: test center `10254` with only one test type and test center `10352` with two test types, and both have minimal number of records given their number of test types.

In [9]:
selected_centers = [10254] # [10254, 10352]

In [10]:
df = df.loc[df['id'].apply(lambda x: x in selected_centers), :]

We will also inspect one particular appointment:

In [11]:
selected_appointment = datetime.datetime(2019, 1, 17, 11, 30)

In [13]:
df[df['appointment'] == selected_appointment].head()

,id,test type,grab,appointment
213759,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00
267453,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00
321128,10254,Blood Test,2019-01-15 14:30:00,2019-01-17 11:30:00
374779,10254,Blood Test,2019-01-15 14:40:00,2019-01-17 11:30:00
428669,10254,Blood Test,2019-01-15 14:50:00,2019-01-17 11:30:00


## Recording all `grab` and `appointment` timestamps
We will shortly need to collect all the `grab` and `appointment` timestamps.

In [14]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
grabs = pd.DataFrame()
grabs['grab'] = df['grab'].drop_duplicates().reset_index(drop=True).sort_values(ascending=True)
#first_grab = grabs.iloc[0, 0 ]
#last_grab = grabs.iloc[-1, 0]
#first_grab, last_grab
#grabs.head()
#grabs.info()

Tuesday 05 February 2019, 22:44:28
CPU times: user 2.89 ms, sys: 1.58 ms, total: 4.47 ms
Wall time: 2.86 ms


In [15]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
appointments = pd.DataFrame()
appointments['appointment'] = df['appointment'].drop_duplicates().reset_index(drop=True).sort_values(ascending=True)
#first_appointment = appointments.iloc[0, 0]
#last_appointment = appointments.iloc[-1, 0]

Tuesday 05 February 2019, 22:44:35
CPU times: user 3.23 ms, sys: 757 µs, total: 3.99 ms
Wall time: 3.05 ms


It will be convenient to crete the full list of (`grab`, `appointment`) pairs where the `grab` timestamp does not come after the `appointment` timestamp.

In [16]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
schedule = cartesianProduct(grabs, appointments)
schedule.query('grab <= appointment', inplace=True)
#schedule.sort_values(['grab', 'appointment'])[1000:1020]

Tuesday 05 February 2019, 22:44:43
CPU times: user 26.1 ms, sys: 4.62 ms, total: 30.7 ms
Wall time: 29.2 ms


In [ ]:
schedule.info()

In [17]:
schedule[schedule['appointment'] == selected_appointment].head()

,grab,appointment
1,2019-01-15 13:40:00,2019-01-17 11:30:00
278,2019-01-15 13:50:00,2019-01-17 11:30:00
555,2019-01-15 14:00:00,2019-01-17 11:30:00
832,2019-01-15 14:10:00,2019-01-17 11:30:00
1109,2019-01-15 14:20:00,2019-01-17 11:30:00


# Reconstruct *entire* booking history (of *all* appointments in *all* centers)?

Duplicate `schedule`, one for each `id`: 

In [18]:
%%time 
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
schedule_duplicate = pd.concat(
    [schedule.assign(id=id_item).assign(type=type_item)
     for id_item in list(df['id'].unique())
     for type_item in list(df['test type'].unique())
     ]
    )

Tuesday 05 February 2019, 22:44:54
CPU times: user 5.03 ms, sys: 1.88 ms, total: 6.91 ms
Wall time: 4.87 ms


In [19]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
schedule_duplicate.rename(index=str, columns={'type': 'test type'}, inplace=True)

Tuesday 05 February 2019, 22:44:56
CPU times: user 6.11 ms, sys: 836 µs, total: 6.95 ms
Wall time: 6.29 ms


In [20]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
schedule_duplicate = schedule_duplicate[['id', 'test type', 'grab', 'appointment']]

Tuesday 05 February 2019, 22:45:05
CPU times: user 1.27 ms, sys: 70 µs, total: 1.34 ms
Wall time: 1.3 ms


In [21]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
schedule_duplicate.sort_values(['id', 'test type', 'grab', 'appointment'],
    inplace=True
)

Tuesday 05 February 2019, 22:45:12
CPU times: user 29.4 ms, sys: 16 ms, total: 45.3 ms
Wall time: 43.5 ms


In [22]:
schedule_duplicate[schedule_duplicate['appointment'] == selected_appointment].head()

,id,test type,grab,appointment
1,10254,Blood Test,2019-01-15 13:40:00,2019-01-17 11:30:00
278,10254,Blood Test,2019-01-15 13:50:00,2019-01-17 11:30:00
555,10254,Blood Test,2019-01-15 14:00:00,2019-01-17 11:30:00
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00
1109,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00


In [ ]:
df0_cart = schedule_duplicate[schedule_duplicate['appointment'] == selected_appointment]

In [ ]:
df0 = df[df['appointment'] == selected_appointment]

In [ ]:
df0.assign(key=1).head()

In [25]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
df_compare = pd.merge(
    schedule_duplicate,
    df.assign(key=1),
    on=['id', 'test type', 'appointment', 'grab'],
    how='outer'
)

Tuesday 05 February 2019, 22:46:48
CPU times: user 12.2 ms, sys: 2.85 ms, total: 15.1 ms
Wall time: 13.6 ms


In [26]:
df_compare[df_compare['appointment'] == selected_appointment].head()

,id,test type,grab,appointment,key
1,10254,Blood Test,2019-01-15 13:40:00,2019-01-17 11:30:00,NaN
278,10254,Blood Test,2019-01-15 13:50:00,2019-01-17 11:30:00,NaN
555,10254,Blood Test,2019-01-15 14:00:00,2019-01-17 11:30:00,NaN
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00,1.0
1109,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00,1.0


In [27]:
#%%time
#print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
#df_compare = pd.merge(
#    df.assign(key=1),
#    schedule_duplicate,
#    how='outer',
#    #how='right',
#    #left_on=['grab', 'appointment', 'id', 'test type'],
#    #right_on=['grab', 'appointment', 'id', 'test type']
##    on=['id', 'test type', 'appointment', 'grab']
#    #on=['appointment', 'grab']
#)

In [28]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))

df_compare.fillna(0, inplace=True)

Tuesday 05 February 2019, 22:47:23
CPU times: user 3.11 ms, sys: 804 µs, total: 3.91 ms
Wall time: 3.06 ms


In [29]:
df_compare[df_compare['appointment'] == selected_appointment].head()

,id,test type,grab,appointment,key
1,10254,Blood Test,2019-01-15 13:40:00,2019-01-17 11:30:00,0.0
278,10254,Blood Test,2019-01-15 13:50:00,2019-01-17 11:30:00,0.0
555,10254,Blood Test,2019-01-15 14:00:00,2019-01-17 11:30:00,0.0
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00,1.0
1109,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00,1.0


In [30]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))
df_compare['key'] = df_compare['key'].astype(int)

Tuesday 05 February 2019, 22:47:57
CPU times: user 716 µs, sys: 90 µs, total: 806 µs
Wall time: 718 µs


In [31]:
df_compare[df_compare['appointment'] == selected_appointment].head()

,id,test type,grab,appointment,key
1,10254,Blood Test,2019-01-15 13:40:00,2019-01-17 11:30:00,0
278,10254,Blood Test,2019-01-15 13:50:00,2019-01-17 11:30:00,0
555,10254,Blood Test,2019-01-15 14:00:00,2019-01-17 11:30:00,0
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00,1
1109,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00,1


In [32]:
%%time
print(datetime.datetime.now().strftime("%A %d %B %Y, %H:%M:%S"))

df_action = df_compare.groupby(['id', 'test type', 'appointment']).apply(spot_action)

Tuesday 05 February 2019, 22:48:12
CPU times: user 485 ms, sys: 24.5 ms, total: 510 ms
Wall time: 491 ms


In [34]:
df_action[df_action['appointment'] == selected_appointment].head()

,id,test type,grab,appointment,key
1,10254,Blood Test,2019-01-15 13:40:00,2019-01-17 11:30:00,none
278,10254,Blood Test,2019-01-15 13:50:00,2019-01-17 11:30:00,none
555,10254,Blood Test,2019-01-15 14:00:00,2019-01-17 11:30:00,none
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00,book
1109,10254,Blood Test,2019-01-15 14:20:00,2019-01-17 11:30:00,none


In [35]:
df_history = df_action.loc[df_action['key'] != "none", :]

In [ ]:
df_history.info()

In [39]:
df_history[df_history['appointment'] == selected_appointment]

,id,test type,grab,appointment,key
832,10254,Blood Test,2019-01-15 14:10:00,2019-01-17 11:30:00,book
22161,10254,Blood Test,2019-01-16 10:50:00,2019-01-17 11:30:00,cancel


=====================================================================================

=====================================================================================

## Selecting a test center
As a warmup, we will select center with only one test type and lowest number of records for speed.

In [ ]:
cid = 10254
df0 = df.loc[df['id'] == cid, ['appointment', 'grab']]
df0.reset_index(drop=True, inplace=True)
#df0.head()

In [ ]:
df0.info()

## Selecting an appointment

As discussed in the other notebook `EDA_2.ipynb`, the 11:30 appointment on 17 Jan 2019 has been cancelled.

In [ ]:
ts_selected = datetime.datetime(2019, 1, 17, 11, 30)

We will find in the dataframes when the appointment was cancelled and in fact when it was rebooked.  
**Caveat.**  *We will miss pairs of cancellation+booking or booking+cancellation if they occur between two grabs.*

First extract the `grab` timestamps for this particular appointment.

In [ ]:
df00 = df0.loc[df0['appointment'] == ts_selected, ['grab']]
df00.head()

# Reconstructing the booking history of the selected appointment at the selected center

We need to compare the `grab` timestamps of the selected appointment with the complete list of `grab` values present in the dataset and recorded in `grabs`.

In [ ]:
df00_compare = compareDataframes(grabs, df00)
df00_compare.index.names = ['timestamp']
df00_compare.head()

With the comparison dataframe `df00_compare`, we can detect bookings and cancellations:

In [ ]:
df00_action = df00_compare.loc[:, 'present_2'].to_frame('present')
df00_action.columns = ['action']
df00_action.iloc[:, 0] = df00_action.iloc[:, 0] - df00_action.iloc[:, 0].shift(1)
df00_action.iloc[:, 0].fillna(method='bfill', inplace=True)
df00_action.iloc[:, 0] = df00_action.iloc[:, 0].astype(int)
df00_action['action'] = df00_action['action'].apply(lambda delta: "cancel" if delta == -1 else ("book" if delta == 1 else "none"))
df00_action.head()

We are now ready to extract the booking history of the isolated appointment.

In [ ]:
df00_history = df00_action.loc[df00_action['action'] != "none", ['action']]
df00_history

# Reconstruct the booking history of *all* appointments (for the selected center)

First we generate the Cartesian product of the `grab` and `appointment` timestamp dataframes.

In [ ]:
df_cartesian = cartesianProduct(appointments, grabs)
df_cartesian.columns = ['appointment', 'grab']
df_cartesian.info()

Visualize slice of Cartesian product for the selected appointment:

In [ ]:
df_cartesian.loc[df_cartesian['appointment'] == ts_selected, :].head()

Some of these records will be missing from the records for the isolated appointment.

In [ ]:
df0.loc[df0['appointment'] == ts_selected, :].head()

Comparing these two dataframes, we will be able to identify when an appoinment was booked or cancelled.  Specifically we may compare the `grab` timestamps for all appointments with the full list of `grab` timestamps in `grabs` .

In [ ]:
df0_ones = pd.DataFrame([1]*df0.shape[0], columns=['key'])
df0_ext = pd.concat([df0, df0_ones], axis=1)  # extend df0 with a column of 1's
df0_ext.info()

Compare `df0_ext` with `df_cartesian` to detect missing (`appointment`, `grab`) pairs:

In [ ]:
df0_compare = pd.merge(df_cartesian, df0_ext, on=['appointment', 'grab'], how='outer')
df0_compare['key'].fillna(0, inplace=True)
df0_compare['key'] = df0_compare['key'].astype(int)
df0_compare.info()

For instance, consider the records corresponding to the selected center and the selected appointment:

In [ ]:
df0_compare.loc[df0_compare['appointment'] == ts_selected, :].head()

The last column `key` indicates whether an appointment was available (with a `1`) at the indicated `grab` timestamp, or not (with a `0`).  

With this we determine the `action` at each `grab` time on the `appointment`: `book` if the appointment was booked at that `grab` time, `cancel` if it was cancelled, and `none` if the appointment was still available at the next `grab` time.

In [ ]:
df0_action = df0_compare.groupby('appointment').apply(spot_action)

For instance, consider the records corresponding to the selected center and the selected appointment:

In [ ]:
df0_action[df0_action['appointment'] == ts_selected].head()

Selecting "book" and "cancel" gives the booking (and cancellation) history of all appointments for the selected center.

In [ ]:
df0_history = df0_action.loc[df0_action['key'] != "none", df0_action.columns]
df0_history.head()

Add "anticipation", that is, the amount of time between booking or cancellation and actual appointment.

In [ ]:
df0_history['anticipation'] = df0_history['appointment'] - df0_history['grab']
df0_history[df0_history['appointment'] == ts_selected].head()

## Visualize booking history

Separate bookings and cancellations.  
**TODO:** achieve the same but without splitting into two dataframes.

In [ ]:
df0_bookings = df0_history.loc[df0_history['key'] == 'book', ['appointment', 'grab', 'anticipation']]
df0_cancellations = df0_history.loc[df0_history['key'] == 'cancel', ['appointment', 'grab', 'anticipation']]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4));
df0_bookings.plot(x='appointment', y='grab', style='.b', ax=ax);
df0_cancellations.plot(x='appointment', y='grab', style='.r', ax=ax);
ax.set_xlabel("appointment date/time");
ax.set_ylabel("booking/cancellation date/time");
ax.get_legend().remove()
ax.set_title("Booking & cancellation history for center {}".format(cid), fontsize=18);

The following to save as `png` image file.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4));
df0_bookings.plot(x='appointment', y='grab', style='.b', ax=ax);
df0_cancellations.plot(x='appointment', y='grab', style='.r', ax=ax);
ax.set_xlabel("appointment");
ax.set_ylabel("booking/cancellation");
ax.legend(['booking', 'cancellation'])
#ax.get_legend().remove()
ax.set_title("Booking history for center {}".format(cid), fontsize=18);
#plt.savefig('center-10254-history.png')

# Visualizing "anticipation"
That is, amount of time between booking or cancellation and actual appointment.

In [ ]:
df0_history.head()

In [ ]:
df0_history[df0_history['appointment'] == ts_cancelled].head()

In [ ]:
def y_func_days(y, pos):
    """Format for labels on y-axis when data is of type pd.Timedelta.  Unit is day."""
    return str(pd.to_timedelta(y).days)
y_fmt_days = tkr.FuncFormatter(y_func_days)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 4));

df0_bookings.plot(x='appointment', y='anticipation', rot=45, figsize=(16, 6), style=':ob', ax=ax)
df0_cancellations.plot(x='appointment', y='anticipation', rot=45, figsize=(16, 6), style=':or', ax=ax)

ax.yaxis.set_major_formatter(y_fmt_days)
ax.set_ylabel("anticipation (days)");
ax.get_legend().remove()
ax.set_title("Anticipation prior to appointment time", fontsize=18);

=======================================================================================

Helper function

In [ ]:
def compareDataframes(df1, df2):
    """Compares two dataframes.
    
    Parameters
    ----------
    df1, df2: dataframes with one column
    
    Returns
    -------
    result: dataframe with index the values of df1 and df2 combined,
    and one column for each of df1 and df2,
    indicating whether the value is present (with a "1")
    or not (with a "0")
    
    """
    df1 = df1.copy()
    df1['value'] = df1.iloc[:, 0]
    df1.set_index('value', inplace=True)
    df2 = df2.copy()
    df2['value'] = df2.iloc[:, 0]
    df2.set_index('value', inplace=True)
    result = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
    result = result.applymap(lambda x: 0 if pd.isnull(x) else 1)
    result.columns = ['present_1', 'present_2']
    return result